In [ ]:
%matplotlib notebook
%autoreload 2
import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as plt
from lightsim.lightsim import LightsimConfig, LightSource

In [ ]:
cfg = LightsimConfig()
etch_pattern = np.zeros((512, 512), dtype=np.bool)
etch_pattern[128:-128, :] = True
etch_pattern[:, 128:-128] = True
cfg.etch_pattern = etch_pattern

N = 100000

In [ ]:
ls = LightSource(cfg, 15, 0, 0, np.pi)
dirs_x, dirs_y = ls._random_direction_vectors(N)
dists, offsets = ls._random_angles(N)
min_dist = 2 * np.cos(cfg.alpha_max)
max_hops = np.ceil(np.sqrt(cfg.width**2 + cfg.length**2) / min_dist) + 1
offsets_x = dirs_x * offsets
offsets_y = dirs_y * offsets
hops_x = np.dot(np.arange(max_hops)[:, np.newaxis], dirs_x[np.newaxis, :]) + offsets_x[np.newaxis, :]
hops_y = np.dot(np.arange(max_hops)[:, np.newaxis], dirs_y[np.newaxis, :]) + offsets_y[np.newaxis, :]
coord_x = np.round(hops_x / cfg.width * ls.hits.shape[0]).astype(np.int) + ls.pos_x / cfg.width * ls.hits.shape[0]
coord_y = np.round(hops_y / cfg.length * ls.hits.shape[1]).astype(np.int) + ls.pos_y / cfg.length * ls.hits.shape[1]

for n in range(coord_x.shape[1]):
    cx = np.squeeze(coord_x[:, n]).astype(np.int)
    cy = np.squeeze(coord_y[:, n]).astype(np.int)
    valid = np.ones_like(cx, dtype=np.bool)
    valid[cx < 0] = False
    valid[cx >= ls.hits.shape[0]] = False
    valid[cy < 0] = False
    valid[cy >= ls.hits.shape[1]] = False
    vx = cx[valid]
    vy = cy[valid]
    hits = cfg.etch_pattern[vx, vy]
    if np.any(hits):
        hidx = np.argmax(hits)
        ls.hits[vx[hidx], vy[hidx]] = ls.hits[vx[hidx], vy[hidx]] + 1

In [ ]:
plt.figure()
plt.imshow(etch_pattern, cmap=plt.cm.gray)
#plt.xlim(-10, 522)
#plt.ylim(-10, 522)
plt.hold(True)
plt.plot(256, 0, 'bo')
plt.plot(vcoord_x.flatten(), vcoord_y.flatten(), 'rx', alpha=0.05)

In [ ]:
plt.figure()
plt.imshow(ls.hits > 0, cmap=plt.cm.gray)

In [ ]:
plt.figure()
plt.plot(ls.hits.flatten(), 'x')